In [147]:
import pandas as pd

import requests as req

class ListUnwrapper:
    
    def __init__(self):
        pass
    
    def unwrap_list(self, data_series, col):
        data_list = []
        for data in data_series:
            data_dict = {}
            if data is not None:
                if(type(data) == list and len(data) > 1):
                    for i in range(len(data)):
                        data_dict[str(col)+"."+str(i)] = data[i]
                elif(type(data) == list and len(data) == 0):
                    data_dict[str(col)] = "None"
                data_list.append(data_dict)
        return data_list

class DictUnwrapper:
    
    def __init__(self):
        pass
    
    def unwrap_dict(self, data_series, col):
        data_list = []
        for data in data_series:
            data_dict = {}
            if data is not None:
                if(type(data) == dict):
                    for key, values in data.items():
                        data_dict[str(col)+"."+str(key)] = values
                data_list.append(data_dict)
        return data_list


class JSONUnwrapper(ListUnwrapper, DictUnwrapper):
    
    def __init__(self, url):
        
        super(JSONUnwrapper, self)
        self._url = url
        self._json_data_frame = None
        
    def get_url(self):
        return self._url
    
    def set_url(self, url):
        self._url = url
        
    def get_json_data_frame(self):
        return self._json_data_frame
    
    def set_json_data_frame(self, json_data_frame):
        self._json_data_frame = json_data_frame
        
    def get_json_data(self):
        
        json_obj = JSONExtractor(self.get_url())
        return json_obj.get_json_data()
    
    def json_unwrapper(self):
        
        try:
            
            json_data_frame = pd.DataFrame(self.get_json_data())
            frame_list = []
            frame_list.append(json_data_frame)
            for col in json_data_frame.keys():
                for data in json_data_frame[col]:
                    if data is not None:
                        if(type(data) == dict):
                            temp_frame = pd.DataFrame(self.unwrap_dict(json_data_frame[col], col))
                            frame_list.append(temp_frame)
                            break
                        elif(type(data) == list):
                            temp_frame = pd.DataFrame(self.unwrap_list(json_data_frame[col], col))
                            frame_list.append(temp_frame)
                            break
            return pd.concat(frame_list, axis=1)
            
        except Exception as e:
            
            print("Error occured due to: {} ".format(str(e)))
       
    
class JSONExtractor:
    
    def __init__(self, url):
        self._url = url
        self._json_data = None
        
    def get_url(self):
        return self._url
        
    def set_json_data(self, json_resp):
        self._json_data = json_resp
        
    def get_json_data(self):
        
        try:
            
            resp = req.get(self.get_url())
            self.set_json_data(resp.json())
            return self._json_data
            
        except Exception as e:
            
            print("Error occured due to: {} ".format(str(e)))
            

#input
url = 'https://api.github.com/repos/pandas-dev/pandas/issues'
result_obj = JSONUnwrapper(url)
result_obj.json_unwrapper()
result_obj.json_unwrapper().to_csv("Unravelled_DataSet.csv")